In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

Installing geemap ...


In [2]:
import ee
import os
import datetime
import webbrowser
import geemap
import pprint
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

In [3]:
p = lambda js : print(js.getInfo())
os.chdir("/content/drive/MyDrive/Winter_Research")

# Initialization

In [4]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=dMLsNDfFtNDYadmtY_xaURKkgGiLpUoMUAh8G8LLkhI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4G7Vhe6IW8zZYjL75nKBuxq8VRPmlvb4IhjzsMDNrVZRM1vKvvhnw

Successfully saved authorization token.


In [5]:
Roads = ee.FeatureCollection("TIGER/2016/Roads")
filt_roads = Roads.filter(ee.Filter.equals('rttyp', 'I'))

In [ ]:
cali_roi = ee.Geometry.Polygon([[-120.969389, 37.002966], [-120.859363, 37.015929], [-120.244299, 36.327308], [-120.314643, 36.305507]])
mont_roi = ee.Geometry.Polygon(([[-105.471424, 46.667742], [-105.139824, 46.824933], [-104.850869, 46.966114], [-104.739110, 47.108924], [-104.526622, 47.0799892],
                                 [-104.513387, 47.091907], [-104.753080, 47.15443], [-104.868515, 46.978656], [-105.449642, 46.740551], [-105.482337, 46.740551]]))
texas_roi = ee.Geometry.Polygon(([[-101.257649, 30.693489], [-101.163219, 30.709124], [-100.827636, 30.617001], [-100.311412, 30.464848], [-100.304620, 30.474533],
                                 [-100.492954, 30.568137], [-100.773193, 30.630330], [-101.162826, 30.719190], [-101.260066, 30.702354]]))
kansas_roi = ee.Geometry.Polygon([[-102.477304, 39.300792], [-102.059300, 39.344991], [-101.374279, 39.343892], [-101.362868, 39.327653], [-102.046613, 39.321297], [-102.263878, 39.290923], [-102.47749, 39.289864]])
ohio_roi = ee.Geometry.Polygon([[-84.177001, 40.268322], [-84.153089, 40.328922], [-84.153938, 40.574496], [-83.729733, 40.946666],
                                [-83.777204, 40.975344], [-84.208245, 40.731179], [-84.198750, 40.254644]])

In [ ]:
roi = mont_roi

In [ ]:
fr = filt_roads.filterBounds(roi)

In [ ]:
def get_month_slices(start_yr, end_yr):
  dates = []
  months = []
  for i in range(1, 13):
    str_mon = str(i)
    if len(str_mon) == 1:
      str_mon = "0" + str_mon
    months.append(str_mon)
  for yr in range(start_yr, end_yr):
    str_yr = str(yr)
    for mon in months:
      dates.append(str_yr + '-' + mon + '-01')
  dates.append(str(end_yr) + '-01-01')
  return dates

In [ ]:
img_dates_start = get_month_slices(2018, 2019)
more_dates = get_month_slices(2015, 2019)

In [ ]:
imgs_temp = ee.ImageCollection("COPERNICUS/S2").filterDate("2016-01-01", "2019-01-01").filterBounds(roi)
p(imgs_temp.size())
#imgs_harsh_n = imgs_temp.filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 1)
#imgs_harsh_n = imgs_temp

1065


# Cloudless Mask

In [ ]:
roi = roi
START_DATE = '2016-01-01'
END_DATE = '2019-01-01'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [ ]:
x = ee.ImageCollection('COPERNICUS/S2').filterDate(START_DATE, END_DATE).filterBounds(roi)
p(x.size())

1065


In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    p(s2_sr_col.size())
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [ ]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(roi, START_DATE, END_DATE)

591


In [ ]:
p(s2_sr_cld_col_eval.size())

591


In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    #not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
imgs_harsh_n = s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
                             

In [ ]:
p(imgs_harsh_n)

In [ ]:
dts = ymdList(imgs_harsh_n)

In [ ]:
len(dts)

237

In [ ]:
def mosaic_helper(d, newlist):
  d = ee.Date(d)
  newlist = ee.List(newlist)
  filtered = imgs_harsh_n.filterDate(d, d.advance(1, 'day'))
  image = ee.Image(filtered.mosaic())
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist))

In [ ]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    dates = list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())
    return [x[:4] + "-" + x[4:6] + "-" + x[6:] for x in dates]

In [ ]:
dates_to_it = ymdList(imgs_harsh_n)

In [ ]:
dates_to_it = ["2016-07-01"]

In [ ]:
ee.Image.pixelLonLat().projection().getInfo()['crs']

'EPSG:4326'

In [ ]:
imgs_harsh_n.first().select("B4").projection().getInfo()['crs']

'EPSG:32610'

In [ ]:
dates_to_save

[]

In [ ]:
CRS_pixs = imgs_harsh_n.first().select("B4").projection().getInfo()['crs']
CRS_ll = ee.Image.pixelLonLat().projection().getInfo()['crs']
trans_ll = ee.Image.pixelLonLat().projection().getInfo()['transform']
transform = imgs_harsh_n.first().select("B4").projection().getInfo()['transform']
dates_from_list = []
i = 0
l_of_np = []
#dates = list(pd.read_csv("Dates_Clean.csv")['0'])
dates_to_save = []
for date_str in dates_to_it:
  latitude = []
  longitude = []
  dataB4 = []
  dataB3 = []
  dataB2 = []
  date = ee.Date(date_str)
  filtered = imgs_harsh_n.filterDate(date, date.advance(1, 'day'))
  coords = ee.Image.pixelCoordinates(CRS_pixs)
  filtered = filtered.map(lambda img : img.addBands(img.pixelCoordinates(CRS_pixs)))
  print(i)
  if filtered.size().getInfo() != 0:
    def mask(img):
      road_mask = fr.distance(20).reproject(CRS_pixs, transform, None)
      thresh = road_mask.lt(20).reproject(CRS_pixs, transform, None)
      pixels = img.select("B4").gt(0.5)
      img = img.multiply(pixels)
      return img.multiply(thresh)
    pic_mask = filtered.map(mask)
    collectionList = pic_mask.toList(filtered.size())
    collectionSize = collectionList.size().getInfo()
    tot = 0
    for j in range(collectionSize):
        image = ee.Image(collectionList.get(j))
        bands_c = image.select(["B4", "B3", "B2","x", "y"])
        reduced = bands_c.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, crs=CRS_pixs, maxPixels=1e13, scale=10)
        tot_pixs = ee.Number(bands_c.reduceRegion(reducer=ee.Reducer.count(), geometry=roi, scale=10, maxPixels=1e9).get('B4'))
        if tot_pixs.getInfo():
          dataB4.append(np.array((ee.Array(reduced.get("B4")).getInfo())))
          dataB3.append(np.array((ee.Array(reduced.get("B3")).getInfo())))
          dataB2.append(np.array((ee.Array(reduced.get("B2")).getInfo())))
          latitude.append(np.array((ee.Array(reduced.get("x")).getInfo())))
          longitude.append(np.array((ee.Array(reduced.get("y")).getInfo())))
    if len(latitude) > 0:
      tot_lat = np.concatenate(latitude)
      tot_lon = np.concatenate(longitude)
      tot_B4 = np.concatenate(dataB4)
      tot_B3 = np.concatenate(dataB3)
      tot_B2 = np.concatenate(dataB2)
      lat_lon = np.hstack([tot_lon[:, np.newaxis], tot_lat[:, np.newaxis]])
      uni_lat_lon = np.unique(lat_lon, axis=0)
    print(uni_lat_lon.shape[0])
    if uni_lat_lon.shape[0] == 72099:
      lat = []
      lon = []
      B4 = []
      B3 = []
      B2 = []
      for k in range(uni_lat_lon.shape[0]):
        coord = uni_lat_lon[k]
        where = np.where(np.logical_and((coord[0] == tot_lon), (coord[1] == tot_lat)))
        index = where[0]
        index = index[0]
        if index < len(tot_B4) and index < len(tot_B3) and index < len(tot_B2):
          lat.append(coord[1])
          lon.append(coord[0])
          B4.append(tot_B4[index])
          B3.append(tot_B3[index])
          B2.append(tot_B2[index])
        else:
          print(index)
    else:
      print(tot_lat.shape[0], tot_B2.shape[0])
    if uni_lat_lon.shape[0] == 72099:
      pic = np.array([np.array(B4), np.array(B3), np.array(B2), np.array(lon), np.array(lat)])
      dates_to_save.append(date_str)
      print(date_str)
      l_of_np.append(pic)
    else:
      print(uni_lat_lon.shape[0])
  i += 1
list_t = []
for val in l_of_np:
  list_t.append(val.T)
np_list = np.array(list_t)
x = np_list
arrReshaped = x.reshape(x.shape[0], -1)
np.savetxt("Rois/Roi_3/Cloud_Mask_v2.csv", arrReshaped, delimiter=',')
np.savetxt("Rois/Roi_3/Cloud_Mask_v2_dates.csv", np.array(dates_to_save), delimiter=',', fmt='%s')

0
22117
46635 46635
22117
1
92128
196546 196546
92128
2
72099
2016-03-08
3
42967
112722 112722
42967
4
53174
150522 150522
53174
5
69912
190684 190684
69912
6
72099
2016-06-06
7
72099
2016-06-19
8
102900
201192 201192
102900
9
12576
17178 17178
12576
10
56564
82499 82499
56564
11
69681
197538 197538
69681
12
96646
202374 202374
96646
13
7375
13909 13909
7375
14
46340
127613 127613
46340
15
13371
26632 26632
13371
16
72099
2016-09-27
17
72099
2016-11-06
18
71084
294864 294826
71084
19
21973
47550 47550
21973
20
69586
98672 98672
69586
21
29665
31226 31226
29665
22
29049
33438 33438
29049
23
2073
2073 2073
2073
24
58596
87599 87599
58596
25
54115
77053 77053
54115
26
67192
96280 96280
67192
27
1818
1818 1818
1818
28
756
756 756
756
29
64755
83620 83620
64755
30
14166
15817 15817
14166
31
21800
21800 21800
21800
32
71737
100825 100825
71737
33
1626
1626 1626
1626
34
69910
98998 98998
69910
35
46368
50526 50526
46368
36
43376
71310 71310
43376
37
53551
78364 78364
53551
38
72099
2017-07-19

In [ ]:
len(l_of_np)

23

In [ ]:
plt.scatter(lon, lat)
plt.show()

In [ ]:
display_date("2016-07-01", "2016-07-02")

In [ ]:
def display_date(start, end, mask=False):
  Map = geemap.Map(center=[-120.7601051307836, 36.82972755052831], zoom=50)
  start = ee.Date(start)
  end= ee.Date(end)
  diff = end.difference(start, 'day')
  range = ee.List.sequence(0, diff.subtract(1)).map(lambda day : start.advance(day, 'day'))
  new_col = ee.ImageCollection(ee.List(range.iterate(mosaic_helper, ee.List([]))))
  if mask:  
    pic = new_col.first()
    road_mask = fr.distance(20)
    thresh = road_mask.lt(20)
    new_col = pic.multiply(thresh)
  Map.addLayer(new_col, {
  "bands": ['B4', 'B3', 'B2'], 
  "min": 0,
  "max": 3000,
  })
  return Map


In [ ]:
display_date("2016-07-01", "2016-07-02", mask=True)

In [ ]:
centre

[-120.7601051307836, 36.82972755052831]

# Things to Remember

In [ ]:
station_code = '068510'
data = []
dates_pixels = {}
count = 0
for year in ['2016', '2017', '2018']:
  for month in ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV','DEC']:
    with open("/content/drive/MyDrive/Winter_Research/Cali/CA_"+ month + "_"+ str(year) +" (TMAS).VOL") as fp:
      for line in fp:
        if line[5:11] == station_code:
          count += 1
          data.append(line[20:])
          date = '20' + line[13:15] + '-' + line[15:17] + '-' + line[17:19]
          if date not in dates_pixels:
            dates_pixels[date] = 0
          dates_pixels[date] += int(line[75:80])
print(count)

In [ ]:
# Some Masking syntax:


    #red_pixs = red_ratio.updateMask(red_ratio.gt(red_r)).reduceRegion(reducer=ee.Reducer.count(), geometry=roi, scale=10, maxPixels=1e9).get('B4')
    #green_pixs = green_ratio.updateMask(green_ratio.gt(green_r)).reduceRegion(reducer=ee.Reducer.count(), geometry=roi, scale=10, maxPixels=1e9).get('B3')
    #blue_mean = blue.mean()
    #return img
    #return ee.Image(pic_mask)
    #return ee.List(lst.add(pic_mask))
    #blue_mean = ee.Number(blue_ratio.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=10, maxPixels=1e9).get('B2'))
    #mean_check = ee.Number(red.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=10, maxPixels=1e9))
    #return ee.Number(mean_check)
    #blue_std = ee.Number(blue_ratio.reduceRegion(reducer=ee.Reducer.stdDev(), geometry=roi, scale=10, maxPixels=1e9).get('B2'))